In [2]:
# 导入所需的库
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import os
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [3]:
raw_train_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\train_data.csv")
raw_test_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\test_data.csv")
raw_dev_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\dev_data.csv")

In [4]:
# 删除第一列
train_data = raw_train_data.drop(raw_train_data.columns[0], axis=1)
dev_data = raw_dev_data.drop(raw_dev_data.columns[0], axis=1)
test_data = raw_test_data.drop(raw_test_data.columns[0], axis=1)
t = test_data[test_data['SoH'] != -100]
d = dev_data[dev_data['SoH'] != -100]
combined_df = pd.concat([t, d])
train_data=pd.concat([train_data,combined_df])
# 将值为0的数替换为NaN
train_data.replace(0, np.nan, inplace=True)
test_data.replace(0, np.nan, inplace=True)
train_data=train_data.dropna();
#训练集y
train_y=train_data['SoH']
# #填补缺失值
# train_data['CCCT'] = train_data['CCCT'].fillna(train_data['CCCT'].mean())
# train_data['CVCT'] = train_data['CVCT'].fillna(train_data['CVCT'].mean())
train_data=train_data.drop(columns=['SoH'])
# 对"XX"列进行独热编码
train_data = pd.get_dummies(train_data, columns=["CS_Name"])
# #充电时长
# train_data['CC/CV']=train_data['CCCT']/train_data['CVCT']
# col_to_multiply = {'cycle','CS_Name_CS2_35','CS_Name_CS2_36','CS_Name_CS2_37','CS_Name_CS2_38'}
# # 获取除了 col_to_multiply 列之外的所有列
# cols_to_multiply_with = train_data.drop(col_to_multiply, axis=1).columns
# # 循环相乘
# for col in cols_to_multiply_with:
#     new_col_name = f"cycle_mul_{col}"
#     train_data[new_col_name] = train_data['cycle'] * train_data[col]
#
# train_data['log_cycle'] = np.log(train_data['cycle'])
# train_data['2_cycle'] = train_data['cycle']**2

In [5]:
t_data = test_data[test_data['SoH'] == -100]
#填补缺失值
t_data['CCCT'] = t_data['CCCT'].fillna(t_data['CCCT'].mean())
t_data['CVCT'] = t_data['CVCT'].fillna(t_data['CVCT'].mean())
t_data=t_data.drop(columns=['SoH'])
t_data = pd.get_dummies(t_data, columns=["CS_Name"])

C:\Users\yubin\AppData\Local\Temp\ipykernel_9268\3630547624.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_data['CCCT'] = t_data['CCCT'].fillna(t_data['CCCT'].mean())
C:\Users\yubin\AppData\Local\Temp\ipykernel_9268\3630547624.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_data['CVCT'] = t_data['CVCT'].fillna(t_data['CVCT'].mean())


In [6]:
X_train=train_data;

t_data_x=t_data;

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

X_train_array = X_train.values  # Convert DataFrame to NumPy array

# Reshape and normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train_array.reshape(-1, X_train_array.shape[-1])).reshape(X_train_array.shape)
X_train_tensor = torch.tensor(X_train_scaled)

In [15]:


# Assuming you have a dataset of battery health metrics, X_train and y_train
# X_train shape: (num_samples, sequence_length, num_features)
# y_train shape: (num_samples,)

# Normalize the data
# scaler = MinMaxScaler(feature_range=(0, 1))
# X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)

X_train_array = X_train.values  # Convert DataFrame to NumPy array

# Reshape and normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train_array.reshape(-1, X_train_array.shape[-1])).reshape(X_train_array.shape)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(train_y.values, dtype=torch.float32)

# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Assuming you want only the last output in the sequence
        return out


# Initialize model, loss function and optimizer
input_dim = X_train.shape[-1]
hidden_dim = 64
num_layers = 2
output_dim = 1  # Modify this to sequence_length if predicting a sequence
model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor.to(device))
    loss = criterion(outputs.squeeze(), y_train_tensor.to(device))
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Prediction
model.eval()
with torch.no_grad():
    y_pred = model(X_train_tensor.to(device))
    y_pred = y_pred.cpu().numpy()

# Evaluate the model
# Calculate metrics like RMSE, MAE, etc.


RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors

In [ ]:
y_pred